In [2]:
import pandas as pd
import csv

In [3]:
import sqlite3

conn = sqlite3.connect('books.db')
cursor = conn.cursor()

In [4]:
# create the five tables

# authors table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Authors (
    AuthorID INTEGER PRIMARY KEY,
    AuthorName TEXT NOT NULL
)
''')

# genres table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Genres (
    GenreID INTEGER PRIMARY KEY,
    GenreName TEXT NOT NULL
)
''')
# books table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Books (
    BookID INTEGER PRIMARY KEY,
    Title TEXT NOT NULL,
    AuthorID INTEGER,
    GenreID INTEGER,
    FOREIGN KEY (AuthorID) REFERENCES Authors(AuthorID),
    FOREIGN KEY (GenreID) REFERENCES Genres(GenreID)
)
''')

# editions table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Editions (
    EditionID INTEGER PRIMARY KEY,
    BookID INTEGER,
    YearPublished TEXT,
    EditionLanguage TEXT,
    FOREIGN KEY (BookID) REFERENCES Books(BookID)
)
''')

# ratings table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Ratings (
    RatingID INTEGER PRIMARY KEY,
    BookID INTEGER,
    AverageRating REAL,
    NumberOfRaters INTEGER,
    Score INTEGER,
    NumberOfVoters INTEGER,
    FOREIGN KEY (BookID) REFERENCES Books(BookID)
)
''')

conn.commit()

In [5]:
# extracting from first csv
df = pd.read_csv('best_books.csv')

# inserting authors
for author in df['author'].unique():
    cursor.execute("INSERT INTO Authors (AuthorName) VALUES (?)", (author,))

conn.commit()

for _, row in df.iterrows():
    # get the author's ID
    cursor.execute("SELECT AuthorID FROM Authors WHERE AuthorName = ?", (row['author'],))
    author_id = cursor.fetchone()[0]

    # insert book names
    cursor.execute("INSERT INTO Books (Title, AuthorID) VALUES (?, ?)", (row['book'], author_id))
    book_id = cursor.lastrowid  # Get the ID of the last inserted row

    # insert average rating
    if 'average rating' in df.columns:
        cursor.execute("INSERT INTO Ratings (BookID, AverageRating) VALUES (?, ?)", (book_id, row['average rating']))

conn.commit()

In [6]:
# extracting from second csv
csv_file = 'goodreads_books.csv'

with open(csv_file, 'r', encoding='utf-8') as csv_data:
    csv_reader = csv.DictReader(csv_data)

    for row in csv_reader:
        # insert genres
        cursor.execute("INSERT OR IGNORE INTO Genres (GenreName) VALUES (?)", (row['book_genre'],))
        genre_id = cursor.lastrowid or cursor.execute("SELECT GenreID FROM Genres WHERE GenreName = ?", (row['book_genre'],)).fetchone()[0]

        # insert authors
        cursor.execute("INSERT OR IGNORE INTO Authors (AuthorName) VALUES (?)", (row['author_name'],))
        author_id = cursor.lastrowid or cursor.execute("SELECT AuthorID FROM Authors WHERE AuthorName = ?", (row['author_name'],)).fetchone()[0]

        # insert book names
        cursor.execute("INSERT OR IGNORE INTO Books (Title, AuthorID, GenreID) VALUES (?, ?, ?)", (row['book_name'], author_id, genre_id))
        book_id = cursor.lastrowid or cursor.execute("SELECT BookID FROM Books WHERE Title = ? AND AuthorID = ? AND GenreID = ?", (row['book_name'], author_id, genre_id)).fetchone()[0]

        # insert editions
        cursor.execute("INSERT INTO Editions (BookID, YearPublished, EditionLanguage) VALUES (?, ?, ?)", (book_id, row['year_published'], row['edition_language']))

        # insert ratings
        cursor.execute("INSERT INTO Ratings (BookID, AverageRating, NumberOfRaters, Score, NumberOfVoters) VALUES (?, ?, ?, ?, ?)", 
                       (book_id, row['avg_rating'], row['no_of_raters'], row['score'], row['no_of_ppl_voted']))

conn.commit()

In [7]:
# create price table
cursor.execute('''
CREATE TABLE IF NOT EXISTS BookPrices (
    PriceID INTEGER PRIMARY KEY,
    BookID INTEGER,
    Price TEXT,
    FOREIGN KEY (BookID) REFERENCES Books(BookID)
);
''')

In [ ]:
# calling from nyt api to populate prices table
import sqlite3
import requests
import re

# function to remove content within parentheses and strip leading/trailing whitespace
def clean_title(title):
    return re.sub(r'\s*\([^)]*\)', '', title).strip()

# function to retreive price from API given book title and author
def fetch_book_details(title, author):
    url = 'https://api.nytimes.com/svc/books/v3/lists/best-sellers/history.json'
    params = {
        'title': clean_title(title), 
        'author': author,
        'api-key': 'MzY9XRVPMRNlGaZqMsGGCCiXPkGuFkBY' 
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        for result in data['results']:
            # clean both titles before comparison
            api_title = clean_title(result['title'])
            api_author = result['author']
            
            # compare both title and author
            if api_title.lower() == title.lower() and api_author.lower() == author.lower():
                return result.get('price') 
            
        print(f"No matching results found for title: {title}, author: {author}")
    else:
        print(f"API call failed for title: {title}, author: {author}, status code: {response.status_code}")
    
    return None

# get book titles and authors from database
cursor.execute("SELECT Books.BookID, Books.Title, Authors.AuthorName FROM Books JOIN Authors ON Books.AuthorID = Authors.AuthorID")
books = cursor.fetchall()

for book_id, title, author_name in books:
    # get book price from NYT API
    price = fetch_book_details(title, author_name)

    if price is not None:
        # insert book id and price into the BookPrices table
        cursor.execute("INSERT INTO BookPrices (BookID, Price) VALUES (?, ?)", (book_id, price))

conn.commit()

In [11]:
# sql queries to demonstrate proper functionality 

# query to list books with their authors and genres
cursor.execute('''SELECT Books.Title AS BookTitle,Authors.AuthorName AS Author,Genres.GenreName AS Genre
                    FROM Books
                    JOIN Authors ON Books.AuthorID = Authors.AuthorID
                    JOIN Genres ON Books.GenreID = Genres.GenreID
                    ORDER BY Genres.GenreName, Authors.AuthorName;''')

rows = cursor.fetchall() 
for row in rows:
    print(row)

('Homecoming: The Assiduous Quest of Tobias Hopkins - Part One', 'James Faro', 'Adventure')
('Homecoming: The Assiduous Quest of Tobias Hopkins - Part One', 'James Faro', 'Adventure')
('Walks Away Woman', 'Ki Longfellow', 'Adventure')
('Walks Away Woman', 'Ki Longfellow', 'Adventure')
('The Age of Insight: The Quest to Understand the Unconscious in Art, Mind, and Brain, from Vienna 1900 to the Present', 'Eric R. Kandel', 'Art')
('The Age of Insight: The Quest to Understand the Unconscious in Art, Mind, and Brain, from Vienna 1900 to the Present', 'Eric R. Kandel', 'Art')
('History of Beauty', 'Umberto Eco', 'Art')
('History of Beauty', 'Umberto Eco', 'Art')
("Don't Let's Go to the Dogs Tonight: An African Childhood", 'Alexandra Fuller', 'Autobiography')
("Don't Let's Go to the Dogs Tonight: An African Childhood", 'Alexandra Fuller', 'Autobiography')
('Truth and Beauty', 'Ann Patchett', 'Autobiography')
('Truth and Beauty', 'Ann Patchett', 'Autobiography')
('Running with Scissors', 'Aug

('Black House (The Talisman, #2)', 'Stephen King', 'Horror')
('Hyperbole and a Half: Unfortunate Situations, Flawed Coping Mechanisms, Mayhem, and Other Things That Happened', 'Allie Brosh', 'Humor')
('Hyperbole and a Half: Unfortunate Situations, Flawed Coping Mechanisms, Mayhem, and Other Things That Happened', 'Allie Brosh', 'Humor')
("Are You There, Vodka? It's Me, Chelsea", 'Chelsea Handler', 'Humor')
("Are You There, Vodka? It's Me, Chelsea", 'Chelsea Handler', 'Humor')
('A Dirty Job (Grim Reaper, #1)', 'Christopher Moore', 'Humor')
('You Suck (A Love Story, #2)', 'Christopher Moore', 'Humor')
('A Dirty Job (Grim Reaper, #1)', 'Christopher Moore', 'Humor')
('You Suck (A Love Story, #2)', 'Christopher Moore', 'Humor')
('Dress Your Family in Corduroy and Denim', 'David Sedaris', 'Humor')
('When You Are Engulfed in Flames', 'David Sedaris', 'Humor')
('Dress Your Family in Corduroy and Denim', 'David Sedaris', 'Humor')
('When You Are Engulfed in Flames', 'David Sedaris', 'Humor')
("F

('Mockingbird', 'Kathryn Erskine', 'Young Adult')
('Pushing the Limits (Pushing the Limits, #1)', 'Katie McGarry', 'Young Adult')
('Pushing the Limits (Pushing the Limits, #1)', 'Katie McGarry', 'Young Adult')
('The Sea of Tranquility', 'Katja Millay', 'Young Adult')
('The Sea of Tranquility', 'Katja Millay', 'Young Adult')
('The Summoning (Darkest Powers, #1)', 'Kelley Armstrong', 'Young Adult')
('The Reckoning (Darkest Powers, #3)', 'Kelley Armstrong', 'Young Adult')
('The Awakening (Darkest Powers, #2)', 'Kelley Armstrong', 'Young Adult')
('The Gathering (Darkness Rising, #1)', 'Kelley Armstrong', 'Young Adult')
('The Summoning (Darkest Powers, #1)', 'Kelley Armstrong', 'Young Adult')
('The Reckoning (Darkest Powers, #3)', 'Kelley Armstrong', 'Young Adult')
('The Awakening (Darkest Powers, #2)', 'Kelley Armstrong', 'Young Adult')
('The Gathering (Darkness Rising, #1)', 'Kelley Armstrong', 'Young Adult')
('Anna Dressed in Blood (Anna, #1)', 'Kendare Blake', 'Young Adult')
('Anna Dres

In [12]:
# query to find the top 5 genres with the most ratings
cursor.execute('''SELECT Genres.GenreName,COUNT(Ratings.BookID) AS TotalRatings
                    FROM Ratings
                    JOIN Books ON Ratings.BookID = Books.BookID
                    JOIN Genres ON Books.GenreID = Genres.GenreID
                    GROUP BY Genres.GenreName
                    ORDER BY TotalRatings DESC
                    LIMIT 5;''')

rows = cursor.fetchall() 
for row in rows:
    print(row)

('Fiction', 770)
('Fantasy', 544)
('Young Adult', 352)
('Nonfiction', 234)
('Historical', 220)


In [13]:
# query to calculate average rating of books by each author
cursor.execute('''SELECT Authors.AuthorName,AVG(Ratings.AverageRating) AS AvgRating
                    FROM Ratings
                    JOIN Books ON Ratings.BookID = Books.BookID
                    JOIN Authors ON Books.AuthorID = Authors.AuthorID
                    GROUP BY Authors.AuthorName
                    ORDER BY AvgRating DESC;''')

rows = cursor.fetchall() 
for row in rows:
    print(row)

('A.   White', 4.94)
('Alan Roger Currie', 4.91)
('Robert M. Richburg', 4.86)
('Lydia Petersen', 4.86)
('James W. Perkinson', 4.86)
('Eileen Albrizio', 4.86)
('Anna  Paulsen', 4.86)
('Fred Krebsbach', 4.85)
('Erin Darcy', 4.84)
('Elias Zapple', 4.832000000000002)
("Hadel S. Ma'ayeh", 4.83)
('Aesop Rock', 4.82)
('M.D. Ashouri, Sami', 4.8)
('Elyse Draper', 4.8)
('Ahmad Baari', 4.8)
('Aldrin Mathew', 4.798)
('Valentine,', 4.79)
('Jason R. Richter', 4.79)
('Ramos Talaya', 4.78)
('Hayley Paige', 4.78)
('Dr. Bee', 4.78)
('Crow Johnson', 4.78)
('Melissa Se', 4.77)
('Helene Cardona', 4.77)
('Merlin Waterson', 4.75)
('Germany Kent', 4.75)
('Elgon Williams', 4.75)
('Chögyam Trungpa', 4.75)
('Christian Cavendish', 4.75)
('Bill Watterson', 4.745882352941175)
('Marisa Mohdi', 4.74)
('Diana Rowe', 4.735)
('Morgan Taylor Giesbrecht', 4.73)
('J.G.  Johnson', 4.73)
('Garrett M. Graff', 4.72)
('Yuriy Grigoryants', 4.71)
('MsKingBean89', 4.71)
('Althea S.T.', 4.71)
('Anna Othitis', 4.7025)
('Rich Okun', 

('S.A. Cosby', 4.1)
('Roshani Chokshi', 4.1)
('Reginald Hill', 4.1)
('Ramg Vallath', 4.1)
('Peter L. Bergen', 4.1)
('Peter Boylan', 4.1)
('Patrick Lencioni', 4.1)
('Melanie Joy', 4.1)
('Mac Barnett', 4.1)
('Lily Brooks-Dalton', 4.1)
('Kimberley Freeman', 4.1)
('Kevin Leman', 4.1)
('Katherine Arden', 4.1)
('Jonathan Haidt', 4.1)
('Jonathan Auxier', 4.1)
('John Patrick Shanley', 4.1)
('Jessica  Kim', 4.1)
('Jane Carter Woodrow', 4.1)
("Jamie O'Neill", 4.1)
('James Comey', 4.1)
('Jacques Derrida', 4.1)
('Jacqueline Woodson', 4.1)
('Jacob A. Zumoff', 4.1)
('F.H. Batacan', 4.1)
('Eric Grounds', 4.1)
('Crystal Wilkinson', 4.1)
('Courtney C. Stevens', 4.1)
('Clive Barker', 4.1)
('Christopher Scotton', 4.1)
('Christopher Brookmyre', 4.1)
('Ben Hatke', 4.1)
('Barbara Natterson-Horowitz', 4.1)
('B.L. Purdom', 4.1)
('Astrid Rosenfeld', 4.1)
('Annabelle Sami', 4.1)
('Ammi-Joan Paquette', 4.1)
('Ali Benjamin', 4.1)
('Adam Rex', 4.1)
('Carlos Ruiz Zafón', 4.099999999999999)
('Terry Pratchett', 4.099

('Edoardo Albert', 3.9500000000000006)
('Diane Stanley', 3.9500000000000006)
('David A. Price', 3.9500000000000006)
('Danielle Rohr', 3.9500000000000006)
('Daniel L. Everett', 3.9500000000000006)
('Daniel H. Pink', 3.9500000000000006)
('Clive Cussler', 3.9500000000000006)
('Cassie Chambers', 3.9500000000000006)
('Caroline Kepnes', 3.9500000000000006)
('Amal El-Mohtar', 3.9500000000000006)
('Alison   McQueen', 3.9500000000000006)
('Alex Capus', 3.9500000000000006)
('Adam Cohen', 3.9500000000000006)
('Sheryl Sandberg', 3.95)
('Shirley Damsgaard', 3.9499999999999997)
('Perumal Murugan', 3.9499999999999997)
("Maggie O'Farrell", 3.9499999999999997)
('Kerri Maniscalco', 3.9499999999999997)
('Karen Hawkins', 3.9499999999999997)
('Jack Kerouac', 3.9499999999999997)
('Jonathan Tropper', 3.9481818181818182)
('Rachel Hartman', 3.9479999999999995)
('Lauren Dane', 3.946666666666667)
('Jeff W. Horton', 3.946666666666667)
('Yasmina Khadra', 3.9450000000000003)
('Pete Nelson', 3.9450000000000003)
('Be

('Paul Tremblay', 3.7999999999999994)
('Michael Sims', 3.7999999999999994)
('Kody Keplinger', 3.7999999999999994)
('Kashmira Sheth', 3.7999999999999994)
('Jude Deveraux', 3.7999999999999994)
('John Tayman', 3.7999999999999994)
('John Kerwin', 3.7999999999999994)
('John Corey Whaley', 3.7999999999999994)
('Joel Kraemer', 3.7999999999999994)
('Jim Holt', 3.7999999999999994)
('Jim C. Hines', 3.7999999999999994)
('Jane McGonigal', 3.7999999999999994)
('Isabel Abedi', 3.7999999999999994)
('Howard F. Stein', 3.7999999999999994)
('Gao Xingjian', 3.7999999999999994)
('Felix O. Hartmann', 3.7999999999999994)
('Don Paterson', 3.7999999999999994)
('Deborah Mitford', 3.7999999999999994)
('Craig      Campbell', 3.7999999999999994)
('Courtney Milan', 3.7999999999999994)
('Christina Dodd', 3.7999999999999994)
('Catherine Asaro', 3.7999999999999994)
('Carlos Eire', 3.7999999999999994)
('C.J. Somersby', 3.7999999999999994)
('Brian Keene', 3.7999999999999994)
('Balli Kaur Jaswal', 3.7999999999999994)
('

('Adrian Gray', 3.6)
('Andre Dubus III', 3.5999999999999996)
('D.B.C. Pierre', 3.595999999999999)
('Carrie Ryan', 3.595999999999999)
('Michael Crichton', 3.595789473684211)
('Jill Nelson', 3.595)
('Gabriel García Márquez', 3.594)
('Ian McEwan', 3.5938461538461532)
('Erik Larson', 3.593)
('Yashodhara Lal', 3.59)
('Tomasz Bochiński', 3.59)
('Tim Krabbé', 3.59)
('Susan Krinard', 3.59)
('Sarah Palin', 3.59)
('Sam Kieth', 3.59)
('Roy Huff', 3.59)
('Robert Hillman', 3.59)
('Peter D. Campbell', 3.59)
('Nancy Moser', 3.59)
('Nami Mun', 3.59)
('Meg Wolitzer', 3.59)
('Mathias Malzieu', 3.59)
('Mark Z. Danielewski', 3.59)
('Maggie Knox', 3.59)
('Krista Madden', 3.59)
('Joyce Maynard', 3.59)
('Jesse Ventura', 3.59)
('Helen Callaghan', 3.59)
('Harry G. Frankfurt', 3.59)
('Geoff Ryman', 3.59)
('Daniel H. Wilson', 3.59)
('Daniel       Mason', 3.59)
('Catherine Knutsson', 3.59)
('C.J. Flood', 3.59)
('Brian Minchin', 3.59)
('April Lindner', 3.59)
('Andrew  Miller', 3.59)
('Laini Taylor', 3.588125000000

In [18]:
# query to find edition information for a certain book ('Prince of Thieves')
cursor.execute('''SELECT Editions.EditionID, Editions.YearPublished, Editions.EditionLanguage
                    FROM Editions
                    JOIN Books ON Editions.BookID = Books.BookID
                    WHERE Books.Title = 'Prince of Thieves'
                    ORDER BY Editions.YearPublished;''')

rows = cursor.fetchall() 
for row in rows:
    print(row)

(1500, '2004', 'English')
(3000, '2004', 'English')
